
# Assignment 2: Extended Kalman Filters and Robot Localisation

## By Daniel Eneh
### H00421147
### Solution Explanation
### Task 1
- 1 <code>run roscore</code> on <strong>Terminal 1</strong>
- 2 run the ekf_node.py on <strong>Terminal 2</strong> using <code>rosrun assigment2 ekf_node.py</code>
- 3 Start up rviz by running <code>rosrun rviz rviz</code> on <strong>Terminal 3</strong>
- 4 Ensure only the visual odometry topic is checked and set the colour to <span style="color:red">red</span>
- 5 Start the bag by running <code>rosbag play dataset_ws/assignment2.bag</code>

After the bag was run a screenshot of the trajectory as seen below was taken
- Terminate all runnning terminal using <b>CTRL + C</b>

![](screenshot_motion_Daniel_Eneh.jpg)
### Task 2
- 1 <code>run roscore</code> on <strong>Terminal 1</strong>
- 2 run the ekf_node.py on <strong>Terminal 2</strong> using <code>rosrun assigment2 ekf_node.py</code>
- 3 Start up rviz by running <code>rosrun rviz rviz</code> on <strong>Terminal 3</strong>
- 4 Ensure only the gps topic is checked and set the colour to <span style="color:red">red</span>
- 5 Start the bag by running <code>rosbag play dataset_ws/assignment2.bag</code>
After the bag was run a screenshot of the trajectory as seen below was taken
- Terminate all runnning terminal using <b>CTRL + C</b>

![](screenshot_measurement_Daniel_Eneh.jpg)
### Task 3
The ekf_predict() and ekf_update() were written using the Extended Kalman Filter Algorithm formula as shown in the class note as seen below

![](ekf_formula.jpg)

Where ekf_predict uses equation <b>2 and 3</b> ekf_update uses the equations <b>4,5,6,7 </b>
<br></br>
<br><strong>The Parameters for the assignment are defined as below</strong></br>
![](Models.jpg)
<br></br>
<br></br>
The functions <code>ekf_prediction()</code> and <code>ekf_update</code> are defined as follows

In [ ]:
def ekf_prediction(self):
        # PUT YOUR CODE HERE - FOLLOW EQUATIONS IN NOTES

        # Defining Motion Model for x for state to estimate X = [x,y,yaw]
        Xt = self.Mt[0][0] + np.cos(self.Mt[2][0]) * self.XvelLin  \
            - np.sin(self.Mt[2][0]) * self.YvelLin
        Yt = self.Mt[1][0] + np.sin(self.Mt[2][0]) * self.XvelLin  \
            +  np.cos(self.Mt[2][0]) * self.YvelLin
        Yawt = self.Mt[2][0] + self.ZvelAng

        # Defining the Jacobian Matrix
        G_t = np.array([[1,   0, -(self.XvelLin * np.sin(Yawt))-(self.YvelLin * np.cos(Yawt))],
                        [0,   1,  (self.XvelLin * np.cos(Yawt)) -
                         (self.YvelLin * np.sin(Yawt))],
                        [0,   0, 1]
                        ])

        # Prediction equation
        # Equation 2 from Note
        Mt_pred = np.array([[Xt],
                            [Yt],
                            [Yawt]])
        print('Mt ', Mt_pred)
        # Equation 3 from Note
        sigma_t_pred = (np.dot(np.dot(G_t, self.sigma_t),
                        np.transpose(G_t))) + self.Rt

        self.Mt = Mt_pred
        self.sigma_t = sigma_t_pred

In [ ]:
def ekf_update(self):
        # when new gps data comes in
        # ADD YOUR CODE HERE
        Ht = np.array([[1, 0, 0],
                       [0, 1, 0]
                       ])
        Ht_transpose = np.transpose(Ht)

        # Equation 4 From Note
        # Defining Kalman Gain
        Kt = np.linalg.multi_dot(
            [self.sigma_t,
             Ht_transpose,
             np.linalg.inv(
                 np.linalg.multi_dot(
                     [Ht, self.sigma_t, Ht_transpose]) + self.Qt
             )
             ]
        )
        # Define Zt
        Zt = np.array([[self.Xgps], [self.Ygps]])

        # Equation 5 from Note
        self.Mt = self.Mt + np.dot(Kt,
                                   (Zt - np.dot(Ht, self.Mt))
                                   )

        # Defining 3 x 3 Identity matrix
        I = np.eye(3)

        # Equation 6 from Note
        self.sigma_t = np.dot(I - np.dot(Kt, Ht), self.sigma_t)

        #print("mt", self.Mt)
        #print("sigmat", self.sigma_t)

        print("update done")

To visualise the <code>fuse_pose</code>
- 1 <code>run roscore</code> on <strong>Terminal 1</strong>
- 2 run the ekf_node.py on <strong>Terminal 2</strong> using <code>rosrun assigment2 ekf_node.py</code>
- 3 Start up rviz by running <code>rosrun rviz rviz</code> on <strong>Terminal 3</strong>
- 4 Ensure only the <code>fuse_pose</code> topic is checked and set the colour to <span style="color:green">green</span>
- 5 Start the bag by running <code>rosbag play dataset_ws/assignment2.bag</code> on a new terminal
After the bag was run a screenshot of the trajectory as seen below was taken
- Terminate all runnning terminal using <b>CTRL + C</b>
The fused posed of the Extended Kalman Filter is show in the image below

![](screenshot_fusion_Daniel_Eneh.jpg)

### Task 4
The longitude and lattitude of the map was calculated precisely to locate the start position of the robot on the satellite
<br></br>
Lattitude : <b>51.765457</b>
<br></br>
Longitude : <b>-1.258747</b>
<br></br>
<code>catkin_make</code> was used to make the rviz_satellite package


- 1 <code>run roscore</code> on <strong>Terminal 1</strong>
- 2 run the ekf_node.py on <strong>Terminal 2</strong> using <code>rosrun assigment2 ekf_node.py</code>
- 3 Start up <b>rviz_satellite</b> by running <code>roslaunch rviz_satellite assignment_2.launch</code> on <strong>Terminal 3</strong>
- 4 On the satellite visualization add odometry topic and subscribe to the <code>fused_pose_exp</code> and set the colour to <b><span style="color:green">green</span></b>
- 5 Start the bag by running <code>rosbag play dataset_ws/assignment2.bag</code> on a new terminal



The fused_pose is shown in the image below

![](screenshot_fusion_sat_Daniel_Eneh.jpg)

The combined tracjectory of the gps, visual odometry and the fused pose is seen on the <b>rviz</b> and <b>rviz_satellite</b> respectively as seen below
The fused_pose is represented in <b><span style="color:green">green</span></b>
The visual odometry is represented in <b><span style="color:yellow">yellow</span></b>
while the gps is represented in <b><span style="color:red">red</span></b>
![](screenshot_comp_Daniel_Eneh.jpg)
![](screenshot_comp_sat_Daniel_Eneh.jpg)

- Terminate all runnning terminal using <b>CTRL + C</b>


### Task 5
- 1 Install unidecode on terminal using <code>pip install unidecode</code>
- 2 Navigate to evaluation folder using <code>cd catkin_ws/src/assignment_2/src/evaluation</code>
- 3 run <code>python evaluation.py</code>

<b>The error</b> 
![](error.jpg)
<br> </br>
<b>Ground Truth and fused_pose </b> 
![](Ground_Truth_and_Estimate.jpg)
<br></br>

![](fused_pose_estimate.jpg)

<b>The fused pose estimate in csv</b>
![](fused_pose_csv.jpg)